## Exploring Toronto's Neighborhoods


In [177]:
# Imports
import config
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import geocoder # import geocode

In [178]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [179]:
import json
from pandas.io.json import json_normalize

### Acquiring and preprocessing Toronto's Neighborhood data
source:"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [180]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

In [181]:
soup = BeautifulSoup(page.text, "lxml")

In [182]:
#print(soup.prettify())

In [183]:
table = soup.find_all("table")[0]
df = pd.read_html(str(table))[0]

In [184]:
cols = df.loc[0,:].values
df.columns = cols

In [185]:
df = df.loc[1:,:]

In [186]:
print(df.shape)
df.head(3)

(289, 3)


,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods


### Cleaning and manipulating data to create a functional Postcode index
Problems addressed:
1. removal of "Not Assigned" for Boroughs
2. replacement of "Not Assigned" Neighbourhoods with corresponding (where available) Borough designation
3. multiple Neighbourhoods has same Postcode, resulting in duplication of Postcode. Deduplicate by creating a list of consolidated Neighbourhoods for each unique Postcode

In [187]:
df = df[df.Borough != "Not assigned"]
print(df.shape)
df.head(4)

(212, 3)


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [188]:
not_assigned_neighbourhood = df[df.Neighbourhood == "Not assigned"].index

In [189]:
df.loc[not_assigned_neighbourhood, ["Neighbourhood"]] = df.loc[not_assigned_neighbourhood, ["Borough"]].values

Example of Postcode with multiple listed Neighbourhoods. Such duplication of Postcode deny its use as an index

In [190]:
df[df.Postcode == "M5A"]

,Postcode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [191]:
g = df.groupby(["Postcode"])

Consolidating Neighbourhoods belonging to the same Postcode

In [192]:
post_neigh_dic = {}
for key, dfs in g:
    post_neigh_dic[key] = ', '.join(dfs.Neighbourhood.values)

In [193]:
post_neigh_dic["M5A"]

'Harbourfront, Regent Park'

In [194]:
post_neigh_df = pd.DataFrame.from_dict(post_neigh_dic, orient='index', columns=["Consolidate_Neighbourhoods"])
post_neigh_df.head(4)

,Consolidate_Neighbourhoods
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn


Here is a repesentation of Toronto's consolidated Neighbourhoods. We drop and rename columns accordingly.

In [195]:
post_neigh_df = post_neigh_df.merge(df, left_index=True, right_on="Postcode", how="inner")
post_neigh_df.head(6)

,Consolidate_Neighbourhoods,Postcode,Borough,Neighbourhood
12,"Rouge, Malvern",M1B,Scarborough,Rouge
13,"Rouge, Malvern",M1B,Scarborough,Malvern
28,"Highland Creek, Rouge Hill, Port Union",M1C,Scarborough,Highland Creek
29,"Highland Creek, Rouge Hill, Port Union",M1C,Scarborough,Rouge Hill
30,"Highland Creek, Rouge Hill, Port Union",M1C,Scarborough,Port Union
43,"Guildwood, Morningside, West Hill",M1E,Scarborough,Guildwood


In [196]:
post_neigh_df = post_neigh_df.loc[:, ["Postcode", "Borough", "Consolidate_Neighbourhoods"]].drop_duplicates()
post_neigh_df.rename(columns={"Postcode":"PostalCode", "Consolidate_Neighbourhoods": "Neighbourhoods"}, inplace=True)
post_neigh_df.reset_index(drop=True, inplace=True)

**The final cleaned and pre-processed Toronto's Neighbourhood data table**

In [197]:
post_neigh_df.head(12)

,PostalCode,Borough,Neighbourhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [198]:
print(post_neigh_df.shape)

(103, 3)


### Next we add latitude and longitude data to the Toronto's dataset

In [199]:
postal_codes = post_neigh_df.PostalCode.values

In [200]:
#postat_latlng = {}
#for code in postal_codes:
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(code))
#        postat_latlng[code] = g.latlng
        
# Process timed out
# Using provided csv file instead

In [201]:
geospatial_coordinates = pd.read_csv("Geospatial_Coordinates.csv")

In [202]:
geospatial_coordinates.head(4)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917


In [203]:
torontodf = post_neigh_df.merge(geospatial_coordinates,left_on="PostalCode", right_on="Postal Code", how="inner")

In [204]:
torontodf.head(4)

,PostalCode,Borough,Neighbourhoods,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917


## Segmentation in Toronto's Neighbourhoods

In [205]:
toronto_exclusivedf = torontodf.loc[torontodf.Borough.str.contains("Toronto"), :]
toronto_exclusivedf.reset_index(drop=True, inplace=True)

In [206]:
toronto_exclusivedf.head(3)

,PostalCode,Borough,Neighbourhoods,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572


In [207]:
address = "Toronto, CAN"

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinate of Toronto is lat: {}, long: {}".format(latitude, longitude))

The geographical coordinate of Toronto is lat: 43.6604619, long: -79.3861563


### Local map of Toronto overlayed with Neighbourhoods location data

In [208]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# Adding Neighbourhood markers
for lat, long, borough, neighbourhood in zip(toronto_exclusivedf["Latitude"], toronto_exclusivedf["Longitude"],
                    toronto_exclusivedf["Borough"], toronto_exclusivedf["Neighbourhoods"]):
    label = "{}, {}".format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7).add_to(map_toronto)
map_toronto

In [209]:
VERSION = '20180605' # Foursquare API version

### Exploring first Neighbourhood in dataset

In [210]:
first_neighbourhood = toronto_exclusivedf.Neighbourhoods.loc[0]
first_neighbourhood

'The Beaches'

In [211]:
first_neighbourhood_lat = toronto_exclusivedf.Latitude.loc[0]
first_neighbourhood_long = toronto_exclusivedf.Longitude[0]

In [212]:
print("Latitude and Longitude values of {} are lat: {}, long: {}".format(first_neighbourhood, 
                                                                         first_neighbourhood_lat,
                                                                        first_neighbourhood_long))

Latitude and Longitude values of The Beaches are lat: 43.67635739999999, long: -79.2930312


In [213]:
#toronto_exclusivedf.loc[0, :]

### Get top 100 venues within 500 meters of first Neighbourhood

In [214]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    config.CLIENT_ID, 
    config.CLIENT_SECRET, 
    VERSION, 
    first_neighbourhood_lat, 
    first_neighbourhood_long, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=P1LSIHCPCKFWFNT0PNNLOMARYY2Z2LEOO1HIUVMJP2NLAHPK&client_secret=HKK4YQ2M3U3WHKUBPKE31PT3CB0KBKK2RRCOLXW5KDLTRAAS&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [215]:
results = requests.get(url).json()
#results

In [216]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [217]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(30)

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Upper Beaches,Neighborhood,43.680563,-79.292869
3,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [218]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Exploring all Neighbourhoods in Toronto

In [219]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            config.CLIENT_ID, 
            config.CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [220]:
toronto_venues = getNearbyVenues(names=toronto_exclusivedf.Neighbourhoods, 
                                latitudes=toronto_exclusivedf.Latitude,
                                longitudes=toronto_exclusivedf.Longitude)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [221]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Show number of venues returned for each Neighbourhood

In [222]:
cnt_toronto_venues = toronto_venues.groupby("Neighborhood").count().loc[:, ["Venue"]].rename(columns={"Venue":"Count"})
cnt_toronto_venues.reset_index(drop=False, inplace=True)

In [223]:
cnt_toronto_venues.sort_values("Count", ascending=False)

,Neighborhood,Count
0,"Adelaide, King, Richmond",100
7,"Chinatown, Grange Park, Kensington Market",100
31,St. James Town,100
30,"Ryerson, Garden District",100
16,"First Canadian Place, Underground city",100
14,"Design Exchange, Toronto Dominion Centre",100
10,"Commerce Court, Victoria Hotel",100
19,"Harbourfront East, Toronto Islands, Union Station",100
32,Stn A PO Boxes 25 The Esplanade,95
9,Church and Wellesley,87


### Show unique categories from the returned venues

In [224]:
print("There are {} unique categories".format(len(toronto_venues["Venue Category"].unique())))

There are 238 unique categories


## Cluster and analyze each Neighbourhood

In [225]:
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix="", prefix_sep="")
toronto_onehot.drop(columns="Neighborhood", inplace=True)
toronto_onehot = pd.concat([toronto_venues[["Neighborhood"]], toronto_onehot], axis=1)
#toronto_onehot["Neighborhood"] = toronto_venues.Neighborhood
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape

(1705, 238)

### Group rows by Neighborhood and select mean frequency of occurence for each category

In [226]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,...,0.000000,0.00000,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.00,0.012346
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.000000,0.00000,0.00,0.000000,0.011494,0.011494,0.000000,0.011494,0.00,0.011494


In [227]:
toronto_grouped.shape

(38, 238)

### Show each Neighborhood along with top 5 most popular venues

In [228]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.06
2        Cocktail Bar  0.06
3  Italian Restaurant  0.04
4                Café  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1     Coffee Shop  0.11
2            Café  0.11
3       Pet Store  0.05
4    Climbing Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1     Restaurant  0.06
2            Spa  0.06
3  Burrito Place  0.06
4     Smoke Shop  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0  Airport Terminal  0.14
1    Airport Lounge  0.14
2   Airport

### Take popular venues and place into dataframe

In [229]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [230]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Café,Farmers Market,Pub,Seafood Restaurant,Cheese Shop,Beer Bar,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Burrito Place,Stadium,Bar,Caribbean Restaurant,Furniture / Home Store,Climbing Gym,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden,Pizza Place,Park,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Plane,Airport,Airport Food Court,Airport Gate,Harbor / Marina


### Cluster Toronto Neigborhood

In [173]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [174]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_exclusivedf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_clustermerged = toronto_merged.merge(neighborhoods_venues_sorted, left_on='Neighbourhoods', 
                                             right_on='Neighborhood', how='inner')
#toronto_merged.head() # check the last columns!

In [175]:
toronto_clustermerged.head(2)

,PostalCode,Borough,Neighbourhoods,Postal Code,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,The Beaches,Coffee Shop,Pub,Yoga Studio,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Bakery,Fruit & Vegetable Store,Juice Bar,Spa


In [176]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clustermerged['Latitude'], toronto_clustermerged['Longitude'], toronto_clustermerged['Neighborhood'], toronto_clustermerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters